### download the rolling hindcasts for all GCMs from the Copernicus Climate Datastore

In [19]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
import sys
from subprocess import call 
from shutil import which 
import pathlib
from datetime import datetime
from dateutil.relativedelta import relativedelta
import dateparser
import yaml
from yaml.loader import SafeLoader

In [21]:
from dask import delayed, compute
from dask.diagnostics import ProgressBar

In [22]:
import numpy as np
import pandas as pd

In [23]:
HOME = pathlib.Path.home()

In [24]:
CWD = pathlib.Path.cwd()

In [25]:
CWD

PosixPath('/home/nicolasf/operational/ICU/development/hotspots/code/ICU_Water_Watch/notebooks/C3S')

### import local functions for the downloading of the C3S forecasts 

In [26]:
sys.path.append('../..')

In [27]:
from ICU_Water_Watch import C3S, domains

### parameters for papermill

In [41]:
provider = 'CDS' # should not change
varname = 't2m' # can be in ['tprate','t2m', 'sst']
domain_name = 'C3S_download'
# if one wants to download other reforecasts than the ones corresponding to the current month
lag = 0
# path where to save the hindcasts / reforecasts
gcm_path = f'/media/nicolasf/END19101/ICU/data/{provider}'
config_path = '.' # where to find the yaml file(s) mapping GCM to System number
stream = 'hindcasts' # can be 'operational' or 'hindcasts', with the latter corresponding the GCM system with complete hindcasts

### casts the paths to `pathlib.Path` objects 

In [42]:
gcm_path = pathlib.Path(gcm_path)

### expand the path depending on whether we want the 'operational' stream (latest system number) or the 'hindcasts' stream (complete hindcast period)

In [43]:
if stream == 'hindcasts': 
    gcm_path = gcm_path
elif stream == 'operational': 
    gcm_path = gcm_path.joinpath('operational/hindcasts')

In [61]:
print(gcm_path)

/media/nicolasf/END19101/ICU/data/CDS


### reads the yaml file, mapping GCM to system 

Note that the forecast system to system number mapping is available at: 
    
- [https://confluence.ecmwf.int/display/CKB/Description+of+the+C3S+seasonal+multi-system](https://confluence.ecmwf.int/display/CKB/Description+of+the+C3S+seasonal+multi-system)

#### it can be read directly into pandas 

In [44]:
url = 'https://confluence.ecmwf.int/display/CKB/Description+of+the+C3S+seasonal+multi-system'

In [45]:
systems_table = pd.read_html(url)

In [46]:
systems_table = systems_table[1]

In [47]:
systems_table

,Forecasting centre,Forecasting system name,CDS 'system' value
0,ECMWF,System 4,4
1,ECMWF,SEAS5,5
2,Météo-France,System 5,5
3,Météo-France,System 6,6
4,Météo-France,System 7,7
5,Météo-France,System 8,8
6,CMCC,SPS3,3
7,CMCC,SPS3.5,35
8,DWD,GCFS2.0,2
9,DWD,GCFS2.1,21


In [48]:
config_yaml = pathlib.Path(config_path).joinpath(f"CDS_config_{stream}.yaml")

In [49]:
# Open the file and load the file
with open(config_yaml) as f:
    dict_systems = yaml.load(f, Loader=SafeLoader)

In [50]:
dict_systems

{'ECMWF': 5,
 'UKMO': 600,
 'METEO_FRANCE': 8,
 'DWD': 21,
 'CMCC': 35,
 'NCEP': 2,
 'JMA': 3,
 'ECCC_CanCM4i': 2,
 'ECCC_GEM5_NEMO': 3}

In [51]:
dict_systems.keys()

dict_keys(['ECMWF', 'UKMO', 'METEO_FRANCE', 'DWD', 'CMCC', 'NCEP', 'JMA', 'ECCC_CanCM4i', 'ECCC_GEM5_NEMO'])

### get today's date 

In [52]:
date = dateparser.parse('today')

In [53]:
date

datetime.datetime(2022, 9, 23, 9, 34, 42, 625458)

### apply lag 

In [54]:
date = date - relativedelta(months=lag)

In [55]:
print(f"the hindcast data will be downloaded for all re-forecasts initialised in {date:%B}")

the hindcast data will be downloaded for all re-forecasts initialised in September


### generate the dates for the download of the rolling hindcasts 

In [56]:
dates = [datetime(y, date.month, 1) for y in range(1993, 2016 + 1)]

In [57]:
dates

[datetime.datetime(1993, 9, 1, 0, 0),
 datetime.datetime(1994, 9, 1, 0, 0),
 datetime.datetime(1995, 9, 1, 0, 0),
 datetime.datetime(1996, 9, 1, 0, 0),
 datetime.datetime(1997, 9, 1, 0, 0),
 datetime.datetime(1998, 9, 1, 0, 0),
 datetime.datetime(1999, 9, 1, 0, 0),
 datetime.datetime(2000, 9, 1, 0, 0),
 datetime.datetime(2001, 9, 1, 0, 0),
 datetime.datetime(2002, 9, 1, 0, 0),
 datetime.datetime(2003, 9, 1, 0, 0),
 datetime.datetime(2004, 9, 1, 0, 0),
 datetime.datetime(2005, 9, 1, 0, 0),
 datetime.datetime(2006, 9, 1, 0, 0),
 datetime.datetime(2007, 9, 1, 0, 0),
 datetime.datetime(2008, 9, 1, 0, 0),
 datetime.datetime(2009, 9, 1, 0, 0),
 datetime.datetime(2010, 9, 1, 0, 0),
 datetime.datetime(2011, 9, 1, 0, 0),
 datetime.datetime(2012, 9, 1, 0, 0),
 datetime.datetime(2013, 9, 1, 0, 0),
 datetime.datetime(2014, 9, 1, 0, 0),
 datetime.datetime(2015, 9, 1, 0, 0),
 datetime.datetime(2016, 9, 1, 0, 0)]

### path to download the rolling hindcasts 

In [58]:
if not(gcm_path.exists()): 
    gcm_path.mkdir(parents=True)

### Now loops over the dates, creates a dictionnary containing the keywords arguments, and pass to dask for parallel download 

In [59]:
for date in dates: 
    
    downloaded_files = []

    # for GCM in dict_systems.keys():
    for GCM in ['UKMO','JMA']: 
        
        if 'ECCC' in GCM: 
            
            GCM_API_name = GCM.split('_')[0]
            
            GCM_path = gcm_path.joinpath(GCM).joinpath(varname.upper())
            
            print(f"processing ECCC, path should be {str(GCM_path)}")
        
            args = dict(GCM=GCM_API_name,
            system=dict_systems[GCM],
            varname=varname,
            year=date.year,
            month=date.month,
            leadtimes=[1, 2, 3, 4, 5, 6],
            opath=GCM_path,
            domain=domains.domains[domain_name],
            file_format='netcdf',
            level='surface',
            max_retry=3)
        
        else: 

            args = dict(GCM=GCM,
            system=dict_systems[GCM],
            varname=varname,
            year=date.year,
            month=date.month,
            leadtimes=[1, 2, 3, 4, 5, 6],
            opath=gcm_path.joinpath(GCM).joinpath(varname.upper()),
            domain=domains.domains[domain_name],
            file_format='netcdf',
            level='surface',
            max_retry=3)
            

        downloaded_files.append(delayed(C3S.download)(**args))

    with ProgressBar():
        downloaded_files = compute(downloaded_files)

[                                        ] | 0% Completed |  0.0s
/media/nicolasf/END19101/ICU/data/CDS/UKMO/T2M/ensemble_seas_forecasts_t2m_from_1993_09_UKMO.netcdf exists already on disk, skipping download and returning path


attempting to download t2m for GCM JMA, for level surface, year 1993, month 9, in netcdf, attempt 1 of 3

[####################                    ] | 50% Completed |  2.0s

2022-09-23 09:34:52,278 INFO Welcome to the CDS
2022-09-23 09:34:52,279 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/seasonal-monthly-single-levels


[####################                    ] | 50% Completed |  2.5s

2022-09-23 09:34:52,797 INFO Request is completed
2022-09-23 09:34:52,799 INFO Downloading https://download-0002-clone.copernicus-climate.eu/cache-compute-0002/cache/data0/adaptor.mars.external-1663818350.4740324-16136-7-ee815047-4613-4c21-8fe0-2143776d6ef5.nc to /media/nicolasf/END19101/ICU/data/CDS/JMA/T2M/ensemble_seas_forecasts_t2m_from_1993_09_JMA.netcdf (719.2K)


[####################                    ] | 50% Completed |  4.5s

...

[####################                    ] | 50% Completed |  4.6s

2022-09-23 09:37:04,971 INFO Request is queued


[####################                    ] | 50% Completed |  5.9s

2022-09-23 09:37:06,277 INFO Request is failed
2022-09-23 09:37:06,278 ERROR Message: no data is available within your requested subset
2022-09-23 09:37:06,280 ERROR Reason:  Request returned no data
2022-09-23 09:37:06,282 ERROR   Traceback (most recent call last):
2022-09-23 09:37:06,285 ERROR     File "/opt/cdstoolbox/cdscompute/cdscompute/cdshandlers/services/handler.py", line 59, in handle_request
2022-09-23 09:37:06,287 ERROR       result = cached(context.method, proc, context, context.args, context.kwargs)
2022-09-23 09:37:06,289 ERROR     File "/opt/cdstoolbox/cdscompute/cdscompute/caching.py", line 108, in cached
2022-09-23 09:37:06,291 ERROR       result = proc(context, *context.args, **context.kwargs)
2022-09-23 09:37:06,294 ERROR     File "/opt/cdstoolbox/cdscompute/cdscompute/services.py", line 124, in __call__
2022-09-23 09:37:06,296 ERROR       return p(*args, **kwargs)
2022-09-23 09:37:06,298 ERROR     File "/opt/cdstoolbox/cdscompute/cdscompute/services.py", line 60, i


failure to download or save /media/nicolasf/END19101/ICU/data/CDS/UKMO/T2M/ensemble_seas_forecasts_t2m_from_2016_09_UKMO.netcdf

[########################################] | 100% Completed |  6.0s


### convert the notebook to HTML 

### Note: this does not work with papermill

In [28]:
# jupyter = which('jupyter')

# nb_name = ipynbname.name()

# nb_name

# cmd = f"{jupyter} nbconvert --to html {nb_name}.ipynb"

# r = call(cmd, shell=True)